In [1]:
from hvec_importers.rws import communicators as com
from hvec_importers.rws import rws
from hvec_importers.rws import helpers as hlp
from hvec_importers.rws import parsers as parse
import dateutil
import pandas as pd
import requests
import warnings
import swifter

In [2]:
com.TIMEOUT

1200

Understanding the raw data structure of data imported from RWS Waterinfo. Goal is to optimise the parsing procedure. The one in ddlpy relies on nested for-loops and is therefore very slow.

Get some data first. A deep dive in the library is necessary to obtain some raw data to experiment on. Do not try this at home!

In [3]:
warnings.simplefilter('ignore')

In [4]:
locations = rws.station_list()

In [5]:
sel = hlp.create_selection_table(
    locations = locations,
    name = 'Yerseke', quantity = 'WATHTE')

In [6]:
sel

,Code,Locatie_MessageID,Coordinatenstelsel,X,Y,Naam,Parameter_Wat_Omschrijving,Compartiment.Code,Compartiment.Omschrijving,Eenheid.Code,Eenheid.Omschrijving,Grootheid.Code,Grootheid.Omschrijving,Hoedanigheid.Code,Hoedanigheid.Omschrijving,Parameter.Code,Parameter.Omschrijving,start,end
0,YE,19396,25831,574394.194893,5.706876e+06,Yerseke,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,OW,Oppervlaktewater,cm,centimeter,WATHTE,Waterhoogte,NAP,t.o.v. Normaal Amsterdams Peil,NVT,Waarde is niet van toepassing,18000101,21001231
1,YERSKE,20583,25831,574616.706917,5.706661e+06,Yerseke,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,OW,Oppervlaktewater,cm,centimeter,WATHTE,Waterhoogte,NAP,t.o.v. Normaal Amsterdams Peil,NVT,Waarde is niet van toepassing,18000101,21001231


In [7]:
code = 'YERSKE'
sel = sel.query('Code == @code')

In [8]:
start = dateutil.parser.parse('1989-01-01')
end = dateutil.parser.parse('1989-12-31')

In [9]:
req = hlp.create_data_request(sel, start, end)

In [10]:
req.keys()

dict_keys(['AquoPlusWaarnemingMetadata', 'Locatie', 'Periode'])

In [11]:
session = requests.session()
res = com.assert_data_available(sel, start, end, session)
res

True

In [12]:
raw = com._get_raw_slice(sel, start, end, session)
session.close()

In [13]:
raw

{'WaarnemingenLijst': [{'Locatie': {'Locatie_MessageID': 20583,
    'Coordinatenstelsel': '25831',
    'X': 574616.706916963,
    'Y': 5706661.48205982,
    'Naam': 'Yerseke',
    'Code': 'YERSKE'},
   'MetingenLijst': [{'Tijdstip': '1989-12-28T03:40:00.000+01:00',
     'Meetwaarde': {'Waarde_Numeriek': 191.0},
     'WaarnemingMetadata': {'StatuswaardeLijst': ['Gecontroleerd'],
      'BemonsteringshoogteLijst': ['-999999999'],
      'ReferentievlakLijst': ['NVT'],
      'OpdrachtgevendeInstantieLijst': ['ZLXXREG_ZEGE'],
      'KwaliteitswaardecodeLijst': ['00']}},
    {'Tijdstip': '1989-12-28T03:50:00.000+01:00',
     'Meetwaarde': {'Waarde_Numeriek': 189.0},
     'WaarnemingMetadata': {'StatuswaardeLijst': ['Gecontroleerd'],
      'BemonsteringshoogteLijst': ['-999999999'],
      'ReferentievlakLijst': ['NVT'],
      'OpdrachtgevendeInstantieLijst': ['ZLXXREG_ZEGE'],
      'KwaliteitswaardecodeLijst': ['00']}},
    {'Tijdstip': '1989-12-28T04:00:00.000+01:00',
     'Meetwaarde': {'Waa

In [14]:
raw.keys()

dict_keys(['WaarnemingenLijst', 'Succesvol'])

In [15]:
raw

{'WaarnemingenLijst': [{'Locatie': {'Locatie_MessageID': 20583,
    'Coordinatenstelsel': '25831',
    'X': 574616.706916963,
    'Y': 5706661.48205982,
    'Naam': 'Yerseke',
    'Code': 'YERSKE'},
   'MetingenLijst': [{'Tijdstip': '1989-12-28T03:40:00.000+01:00',
     'Meetwaarde': {'Waarde_Numeriek': 191.0},
     'WaarnemingMetadata': {'StatuswaardeLijst': ['Gecontroleerd'],
      'BemonsteringshoogteLijst': ['-999999999'],
      'ReferentievlakLijst': ['NVT'],
      'OpdrachtgevendeInstantieLijst': ['ZLXXREG_ZEGE'],
      'KwaliteitswaardecodeLijst': ['00']}},
    {'Tijdstip': '1989-12-28T03:50:00.000+01:00',
     'Meetwaarde': {'Waarde_Numeriek': 189.0},
     'WaarnemingMetadata': {'StatuswaardeLijst': ['Gecontroleerd'],
      'BemonsteringshoogteLijst': ['-999999999'],
      'ReferentievlakLijst': ['NVT'],
      'OpdrachtgevendeInstantieLijst': ['ZLXXREG_ZEGE'],
      'KwaliteitswaardecodeLijst': ['00']}},
    {'Tijdstip': '1989-12-28T04:00:00.000+01:00',
     'Meetwaarde': {'Waa

And here we are. "raw" is a dictionary with two entries. The second is a boolean, indicating if the import was succesfull. 

The good stuff is in "WaarnemingenLijst" but is disturbingly deep nested. To begin with, the value under the key is a list.

Tried a few json flatteners to no avail. Either do not work or give incorrect data. Try sequence of boring pandas methods.

In [16]:
WaarnemingenLijst = pd.json_normalize(raw, record_path = 'WaarnemingenLijst')
WaarnemingenLijst

,MetingenLijst,Locatie.Locatie_MessageID,Locatie.Coordinatenstelsel,Locatie.X,Locatie.Y,Locatie.Naam,Locatie.Code,AquoMetadata.AquoMetadata_MessageID,AquoMetadata.Parameter_Wat_Omschrijving,AquoMetadata.BemonsteringsApparaat.Code,...,AquoMetadata.Typering.Code,AquoMetadata.Typering.Omschrijving,AquoMetadata.Groepering.Code,AquoMetadata.Groepering.Omschrijving,AquoMetadata.WaardeBepalingstechniek.Code,AquoMetadata.WaardeBepalingstechniek.Omschrijving,AquoMetadata.WaardeBepalingsmethode.Code,AquoMetadata.WaardeBepalingsmethode.Omschrijving,AquoMetadata.WaardeBewerkingsmethode.Code,AquoMetadata.WaardeBewerkingsmethode.Omschrijving
0,"[{'Tijdstip': '1989-12-28T03:40:00.000+01:00',...",20583,25831,574616.706917,5.706661e+06,Yerseke,YERSKE,173961,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,NVT,...,NVT,Waarde is niet van toepassing,NVT,Niet van toepassing,NVT,Waarde is niet van toepassing,other:F039,"Rek. gem. waterhoogte over vorige 2,5 en volge...",NVT,Niet van toepassing
1,"[{'Tijdstip': '1989-11-30T09:00:00.000+01:00',...",20583,25831,574616.706917,5.706661e+06,Yerseke,YERSKE,173623,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,NVT,...,NVT,Waarde is niet van toepassing,NVT,Niet van toepassing,NVT,Waarde is niet van toepassing,other:F027,Rekenkundig gemiddelde waarde over vorige 3 en...,NVT,Niet van toepassing


In [17]:
WaarnemingenLijst.columns

Index(['MetingenLijst', 'Locatie.Locatie_MessageID',
       'Locatie.Coordinatenstelsel', 'Locatie.X', 'Locatie.Y', 'Locatie.Naam',
       'Locatie.Code', 'AquoMetadata.AquoMetadata_MessageID',
       'AquoMetadata.Parameter_Wat_Omschrijving',
       'AquoMetadata.BemonsteringsApparaat.Code',
       'AquoMetadata.BemonsteringsApparaat.Omschrijving',
       'AquoMetadata.BemonsteringsMethode.Code',
       'AquoMetadata.BemonsteringsMethode.Omschrijving',
       'AquoMetadata.BemonsteringsSoort.Code',
       'AquoMetadata.BemonsteringsSoort.Omschrijving',
       'AquoMetadata.BioTaxon.Code', 'AquoMetadata.BioTaxon.Omschrijving',
       'AquoMetadata.BioTaxonType', 'AquoMetadata.BioTaxon_Compartiment.Code',
       'AquoMetadata.BioTaxon_Compartiment.Omschrijving',
       'AquoMetadata.Compartiment.Code',
       'AquoMetadata.Compartiment.Omschrijving', 'AquoMetadata.Eenheid.Code',
       'AquoMetadata.Eenheid.Omschrijving', 'AquoMetadata.Grootheid.Code',
       'AquoMetadata.Grootheid.Oms

In [18]:
MetingenLijst = pd.json_normalize(data = raw['WaarnemingenLijst'], record_path = 'MetingenLijst', meta = ['Locatie', 'AquoMetadata'])
MetingenLijst.columns

Index(['Tijdstip', 'Meetwaarde.Waarde_Numeriek',
       'WaarnemingMetadata.StatuswaardeLijst',
       'WaarnemingMetadata.BemonsteringshoogteLijst',
       'WaarnemingMetadata.ReferentievlakLijst',
       'WaarnemingMetadata.OpdrachtgevendeInstantieLijst',
       'WaarnemingMetadata.KwaliteitswaardecodeLijst', 'Locatie',
       'AquoMetadata'],
      dtype='object')

That's better. Some columns are lists of dictionaries. So flattening and keeping what we need only. Starting with location data

In [19]:
MetingenLijst['WaarnemingMetadata.StatuswaardeLijst'].explode()

0          Gecontroleerd
1          Gecontroleerd
2          Gecontroleerd
3          Gecontroleerd
4          Gecontroleerd
              ...       
61149    Ongecontroleerd
61150    Ongecontroleerd
61151    Ongecontroleerd
61152    Ongecontroleerd
61153    Ongecontroleerd
Name: WaarnemingMetadata.StatuswaardeLijst, Length: 61154, dtype: object

In [20]:
%timeit MetingenLijst['Locatie'].apply(pd.Series)

9.85 s ± 961 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
%timeit MetingenLijst['Locatie'].swifter.apply(pd.Series)

Pandas Apply:   0%|          | 0/61154 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/61154 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/61154 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/61154 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/61154 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/61154 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/61154 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/61154 [00:00<?, ?it/s]

11.8 s ± 1 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
%timeit pd.json_normalize(MetingenLijst['Locatie'])

189 ms ± 23.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
